核心原则：
	1.	Home Credit 这类风控数据里，很多缺失值不是“丢了”，而是“业务上不适用 / 没发生”。因此不能一股脑 fillna，否则会抹掉信号。
	2.	对缺失值要区分：
	•	Not applicable（不适用）：保留 NA + 加 indicator
	•	Unknown（未知）：填入 “Unknown” 或分组统计填补
	•	Data issue（数据错误）：纠正 / 截断 / 删除（要写理由）
	3.	异常值处理优先级：
	•	明显不可能（年龄≤0、收入≤0）→ 设为 NA + indicator / 或删除（要写规则）
	•	极端但可能（收入特别高）→ winsorize/clip（分位数截断）
	4.	删除 vs 保留：以“是否影响建模/分析”和“是否是关键信号”决定，删除必须写阈值和理由。

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

PROJECT_ROOT = Path.cwd().parent

DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR = PROJECT_ROOT / "reports"

print("cwd:", os.getcwd())
print("PROJECT_ROOT:", PROJECT_ROOT)
print("RAW_DIR exists:", RAW_DIR.exists())
print("PROCESSED_DIR exists:", PROCESSED_DIR.exists())

cwd: /Users/haiou/Ironhacker/home-credit-risk-analytics/notebooks
PROJECT_ROOT: /Users/haiou/Ironhacker/home-credit-risk-analytics
RAW_DIR exists: True
PROCESSED_DIR exists: True


In [3]:
files = {
    "application_train": RAW_DIR / "application_train.csv",
    "application_test": RAW_DIR / "application_test.csv",
    "bureau": RAW_DIR / "bureau.csv",
    "bureau_balance": RAW_DIR / "bureau_balance.csv",
    "previous_application": RAW_DIR / "previous_application.csv",
    "POS_CASH_balance": RAW_DIR / "POS_CASH_balance.csv",
    "credit_card_balance": RAW_DIR / "credit_card_balance.csv",
    "installments_payments": RAW_DIR / "installments_payments.csv",
}

{k: v.exists() for k, v in files.items()}

{'application_train': True,
 'application_test': True,
 'bureau': True,
 'bureau_balance': True,
 'previous_application': True,
 'POS_CASH_balance': True,
 'credit_card_balance': True,
 'installments_payments': True}

In [ ]:
def missing_summary(df, table_name):
    n = len(df)
    return pd.DataFrame({
        "table": table_name,
        "column": df.columns,
        "dtype": df.dtypes.astype(str),
        "missing_count": df.isna().sum(),
        "missing_pct": (df.isna().sum() / n * 100).round(2),
    }).reset_index(drop=True)

In [ ]:
summaries = []

for name, path in files.items():
    print(f"Loading {name} ...")
    df = pd.read_csv(path)
    summaries.append(missing_summary(df, name))

missing_all = pd.concat(summaries, ignore_index=True)

missing_nonzero = (
    missing_all[missing_all["missing_count"] > 0]
    .sort_values(...)
)

missing_nonzero.head(20)

Loading application_train ...
Loading application_test ...
Loading bureau ...
Loading bureau_balance ...
Loading previous_application ...
Loading POS_CASH_balance ...
Loading credit_card_balance ...
Loading installments_payments ...


,table,column,dtype,missing_count,missing_pct
303,POS_CASH_balance,CNT_INSTALMENT,float64,26071,0.26
304,POS_CASH_balance,CNT_INSTALMENT_FUTURE,float64,26087,0.26
169,application_test,COMMONAREA_AVG,float64,33495,68.72
183,application_test,COMMONAREA_MODE,float64,33495,68.72
197,application_test,COMMONAREA_MEDI,float64,33495,68.72
177,application_test,NONLIVINGAPARTMENTS_AVG,float64,33347,68.41
191,application_test,NONLIVINGAPARTMENTS_MODE,float64,33347,68.41
205,application_test,NONLIVINGAPARTMENTS_MEDI,float64,33347,68.41
207,application_test,FONDKAPREMONT_MODE,object,32797,67.28
175,application_test,LIVINGAPARTMENTS_AVG,float64,32780,67.25


In [6]:
strategy = missing_nonzero.copy()

strategy["missing_reason"] = ""
strategy["action"] = ""
strategy["method"] = ""
strategy["rationale"] = ""

strategy.head(20)

,table,column,dtype,missing_count,missing_pct,missing_reason,action,method,rationale
303,POS_CASH_balance,CNT_INSTALMENT,float64,26071,0.26,,,,
304,POS_CASH_balance,CNT_INSTALMENT_FUTURE,float64,26087,0.26,,,,
169,application_test,COMMONAREA_AVG,float64,33495,68.72,,,,
183,application_test,COMMONAREA_MODE,float64,33495,68.72,,,,
197,application_test,COMMONAREA_MEDI,float64,33495,68.72,,,,
177,application_test,NONLIVINGAPARTMENTS_AVG,float64,33347,68.41,,,,
191,application_test,NONLIVINGAPARTMENTS_MODE,float64,33347,68.41,,,,
205,application_test,NONLIVINGAPARTMENTS_MEDI,float64,33347,68.41,,,,
207,application_test,FONDKAPREMONT_MODE,object,32797,67.28,,,,
175,application_test,LIVINGAPARTMENTS_AVG,float64,32780,67.25,,,,


In [7]:
#handle application_* house related（60%+ missing value）

In [8]:
housing_cols_patterns = [
    "COMMONAREA",
    "LIVINGAPARTMENTS",
    "NONLIVINGAPARTMENTS",
    "FLOORSMIN",
    "YEARS_BUILD",
    "LANDAREA",
]

mask_housing = (
    strategy["table"].isin(["application_train", "application_test"])
    & strategy["column"].apply(lambda x: any(p in x for p in housing_cols_patterns))
)

strategy.loc[mask_housing, ["missing_reason", "action", "method", "rationale"]] = [
    "Not applicable (no property / not assessed)",
    "Keep",
    "Add missing indicator; no direct imputation",
    "High missing rate reflects non-applicability. Missingness itself is informative in credit risk."
]

In [9]:
#hanle OWN_CAR_AGE

In [10]:
mask_car = strategy["column"] == "OWN_CAR_AGE"

strategy.loc[mask_car, ["missing_reason", "action", "method", "rationale"]] = [
    "Client does not own a car",
    "Keep",
    "Add missing indicator; keep NaN",
    "Car ownership is a strong socioeconomic signal; missing implies no car."
]

In [11]:
#handle POS_CASH_balance 

In [12]:
mask_pos = (
    (strategy["table"] == "POS_CASH_balance")
    & strategy["column"].isin(["CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"])
)

strategy.loc[mask_pos, ["missing_reason", "action", "method", "rationale"]] = [
    "Incomplete monthly record / boundary month",
    "Keep",
    "No action at row level; handle in aggregation",
    "POS_CASH_balance is aggregated later; minor missingness handled naturally in stats."
]

In [13]:
strategy_out_path = REPORTS_DIR / "missing_value_strategy_day3.csv"
strategy.to_csv(strategy_out_path, index=False)

strategy_out_path

PosixPath('/Users/haiou/Ironhacker/home-credit-risk-analytics/reports/missing_value_strategy_day3.csv')

In [14]:
prev = pd.read_csv(RAW_DIR / "previous_application.csv")
prev.shape

(1670214, 37)

In [39]:
PLACEHOLDER = 365243

cols_with_placeholder = [
    col for col in prev.columns
    if (prev[col] == PLACEHOLDER).any()
]

id_cols = {"SK_ID_CURR", "SK_ID_PREV"}

cols_to_fix = [c for c in cols_with_placeholder if c not in id_cols]

In [44]:
prev_clean = prev.copy()

for col in cols_to_fix:
  
    prev_clean[f"{col}_is_placeholder"] = (prev_clean[col] == PLACEHOLDER).astype("int8")
    
    prev_clean[col] = prev_clean[col].replace(PLACEHOLDER, np.nan)

In [45]:
for col in cols_to_fix:
    print(col)
    print("  placeholder count:", (prev[col] == PLACEHOLDER).sum())
    print("  NaN after cleaning:", prev_clean[col].isna().sum())
    print("-" * 40)

DAYS_FIRST_DRAWING
  placeholder count: 934444
  NaN after cleaning: 1607509
----------------------------------------
DAYS_FIRST_DUE
  placeholder count: 40645
  NaN after cleaning: 713710
----------------------------------------
DAYS_LAST_DUE_1ST_VERSION
  placeholder count: 93864
  NaN after cleaning: 766929
----------------------------------------
DAYS_LAST_DUE
  placeholder count: 211221
  NaN after cleaning: 884286
----------------------------------------
DAYS_TERMINATION
  placeholder count: 225913
  NaN after cleaning: 898978
----------------------------------------


In [46]:
prev.loc[prev["SK_ID_CURR"] == PLACEHOLDER, ["SK_ID_CURR", "SK_ID_PREV"]].head()

,SK_ID_CURR,SK_ID_PREV
643610,365243,2513618


In [47]:
out_path = PROCESSED_DIR / "previous_application_clean.parquet"
prev_clean.to_parquet(out_path, index=False)

out_path

PosixPath('/Users/haiou/Ironhacker/home-credit-risk-analytics/data/processed/previous_application_clean.parquet')

In [19]:
#Age（DAYS_BIRTH → AGE_YEARS）

In [20]:
app_train = pd.read_csv(RAW_DIR / "application_train.csv")
app_test = pd.read_csv(RAW_DIR / "application_test.csv")

In [21]:
for df in [app_train, app_test]:
    df["AGE_YEARS"] = (-df["DAYS_BIRTH"] / 365.25).round(1)
    df["AGE_OUTLIER"] = ((df["AGE_YEARS"] < 18) | (df["AGE_YEARS"] > 100)).astype("int8")
   
    df.loc[df["AGE_OUTLIER"] == 1, "AGE_YEARS"] = np.nan

In [22]:
app_train[["AGE_YEARS", "AGE_OUTLIER"]].describe()

,AGE_YEARS,AGE_OUTLIER
count,307511.000000,307511.0
mean,43.906968,0.0
std,11.947987,0.0
min,20.500000,0.0
25%,34.000000,0.0
50%,43.100000,0.0
75%,53.900000,0.0
max,69.100000,0.0


In [23]:
#income（AMT_INCOME_TOTAL）

In [24]:
def clean_income(df, lower_q=0.01, upper_q=0.99):
    df = df.copy()
    
    df["INCOME_OUTLIER"] = (df["AMT_INCOME_TOTAL"] <= 0).astype("int8")
    df.loc[df["INCOME_OUTLIER"] == 1, "AMT_INCOME_TOTAL"] = np.nan
    
    lo = df["AMT_INCOME_TOTAL"].quantile(lower_q)
    hi = df["AMT_INCOME_TOTAL"].quantile(upper_q)
    df["AMT_INCOME_TOTAL"] = df["AMT_INCOME_TOTAL"].clip(lo, hi)
    
    return df

app_train = clean_income(app_train)
app_test = clean_income(app_test)

In [25]:
app_train["INCOME_OUTLIER"].value_counts()

INCOME_OUTLIER
0    307511
Name: count, dtype: int64

In [26]:
#credit（AMT_CREDIT）

In [27]:
def clean_credit_amount(df, lower_q=0.01, upper_q=0.99):
    df = df.copy()
    
    df["CREDIT_OUTLIER"] = (df["AMT_CREDIT"] <= 0).astype("int8")
    df.loc[df["CREDIT_OUTLIER"] == 1, "AMT_CREDIT"] = np.nan
    
    lo = df["AMT_CREDIT"].quantile(lower_q)
    hi = df["AMT_CREDIT"].quantile(upper_q)
    df["AMT_CREDIT"] = df["AMT_CREDIT"].clip(lo, hi)
    
    return df

app_train = clean_credit_amount(app_train)
app_test = clean_credit_amount(app_test)

In [28]:
app_train["CREDIT_OUTLIER"].value_counts()

CREDIT_OUTLIER
0    307511
Name: count, dtype: int64

In [29]:
app_train["AMT_INCOME_TOTAL"].describe(percentiles=[0.01, 0.99])

count    307511.000000
mean     166067.479607
std       83000.171936
min       45000.000000
1%        45000.000000
50%      147150.000000
99%      472500.000000
max      472500.000000
Name: AMT_INCOME_TOTAL, dtype: float64

In [30]:
app_train["AMT_CREDIT"].describe(percentiles=[0.01, 0.99])

count    3.075110e+05
mean     5.963060e+05
std      3.913075e+05
min      7.641000e+04
1%       7.641000e+04
50%      5.135310e+05
99%      1.854000e+06
max      1.854000e+06
Name: AMT_CREDIT, dtype: float64

In [31]:
cols = ["AMT_INCOME_TOTAL", "INCOME_OUTLIER", "AMT_CREDIT", "CREDIT_OUTLIER"]

app_test[cols].describe()

,AMT_INCOME_TOTAL,INCOME_OUTLIER,AMT_CREDIT,CREDIT_OUTLIER
count,48744.000000,48744.0,4.874400e+04,48744.0
mean,176627.039053,0.0,5.154275e+05,0.0
std,87007.893231,0.0,3.596129e+05,0.0
min,54000.000000,0.0,6.255450e+04,0.0
25%,112500.000000,0.0,2.606400e+05,0.0
50%,157500.000000,0.0,4.500000e+05,0.0
75%,225000.000000,0.0,6.750000e+05,0.0
max,517500.000000,0.0,1.864152e+06,0.0


In [32]:
app_train.to_parquet(PROCESSED_DIR / "application_train_clean.parquet", index=False)
app_test.to_parquet(PROCESSED_DIR / "application_test_clean.parquet", index=False)

In [33]:
import sys; sys.path.append("..")

In [34]:
from src.data.cleaning import clean_previous_application, clean_application

# quick smoke test
prev = pd.read_csv(RAW_DIR / "previous_application.csv")
prev2 = clean_previous_application(prev)
print(prev.shape, prev2.shape)

app_train = pd.read_csv(RAW_DIR / "application_train.csv")
app_train2 = clean_application(app_train)
print(app_train.shape, app_train2.shape)

(1670214, 37) (1670214, 42)
(307511, 122) (307511, 126)


In [35]:
# previous_application
prev = pd.read_csv(RAW_DIR / "previous_application.csv")
prev_clean = clean_previous_application(prev)
prev_clean.to_parquet(PROCESSED_DIR / "previous_application_clean.parquet", index=False)

# application train/test
app_train = pd.read_csv(RAW_DIR / "application_train.csv")
app_test = pd.read_csv(RAW_DIR / "application_test.csv")

app_train_clean = clean_application(app_train)
app_test_clean = clean_application(app_test)

app_train_clean.to_parquet(PROCESSED_DIR / "application_train_clean.parquet", index=False)
app_test_clean.to_parquet(PROCESSED_DIR / "application_test_clean.parquet", index=False)

print("Day 3 cleaned data saved.")

Day 3 cleaned data saved.


In [36]:
app_train_clean.dtypes.value_counts()
app_train_clean.select_dtypes(include=["category"]).shape

(307511, 16)

In [37]:
raw = pd.read_csv(RAW_DIR / "application_train.csv")
print("raw object cols:", raw.select_dtypes(include=["object"]).shape)

cleaned = app_train_clean
print("cleaned object cols:", cleaned.select_dtypes(include=["object"]).shape)

raw object cols: (307511, 16)
cleaned object cols: (307511, 0)


In [38]:
app_train_fresh = clean_application(raw)
print(app_train_fresh.select_dtypes(include=["object"]).shape)
print(app_train_fresh.select_dtypes(include=["category"]).shape)

(307511, 0)
(307511, 16)


Delete vs Keep rationale
	•	Keep & flag (preferred): For most missing values and abnormal patterns, we keep the rows and add indicator features (e.g., *_OUTLIER, *_is_placeholder). In credit risk, missingness and anomalies often carry signal (e.g., no car → OWN_CAR_AGE missing; placeholder dates → “event did not occur”).
	•	Replace only true technical placeholders: Values like 365243 in previous_application are coded missing values and are converted to NaN to avoid misleading numeric interpretations.
	•	Clip instead of drop for extreme numerics: For AMT_INCOME_TOTAL and AMT_CREDIT, we clip extreme tails (1%–99%) to reduce the influence of outliers while preserving sample size.
	•	Drop only when record is unusable: We would only remove rows when critical identifiers are missing/invalid (e.g., missing SK_ID_CURR) or when a row cannot be linked/aggregated reliably.
	•	Consistency across train/test: The same cleaning and typing rules are applied to both train and test to avoid train–test leakage and ensure stable downstream pipelines.